In [2]:
# move to project root
%cd ..
%pwd

/Users/heste/workspace/soccernet/sn-script


/Users/heste/workspace/soccernet/sn-script/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/heste/workspace/soccernet/sn-script'

In [4]:
import os
import pandas as pd

label_template = "commentary_dataset/gpt-3.5-turbo-1106_500game_{half_nunber}_llm_annotation_{split}.csv"

split = "train"
label_df_half1 = pd.read_csv(
    os.path.join(label_template.format(half_nunber=1, split=split))
)
label_df_half2 = pd.read_csv(
    os.path.join(label_template.format(half_nunber=2, split=split))
)


In [5]:
games_half1 = label_df_half1["game"].unique().tolist()
games_half2 = label_df_half2["game"].unique().tolist()

In [6]:
len(games_half1), len(games_half2), len(set(games_half1) & set(games_half2))

(271, 275, 270)

In [7]:
from SoccerNet.Downloader import getListGames
games = getListGames("train", "caption")

In [8]:
len(games)

281

In [10]:
len(label_df_half1)

132219

In [11]:
label_df_half1 = label_df_half1.dropna(subset=["target_label"])

In [12]:
len(label_df_half1)

131724